## Homework 8 - Training a GCN model on the Cora dataset

Please implement the following three functions:
- normalize_adj() - Normalize adjacent matrix
- GraphConvolution() - Design a graph convolution layer
- GCN() - Construct a graph convolutional Network

Please train a two-layer GCN with hidden dimension of 32 on the Cora dataset and print the training results for each epoch.

In [ ]:
import time
import math
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from utils import load_data, accuracy


def normalize_adj(adj):
    """Normalize adjacent matrix"""
    # Tip: norm_adj
    pass


# Seed radom seed
np.random.seed(42)
torch.manual_seed(42)

# Load dat
adj, features, labels, idx_train, idx_val, idx_test = load_data()
adj = normalize_adj(adj)

In [ ]:
class GraphConvolution(Module):
    """Simple GCN layer, similar to https://arxiv.org/abs/1609.02907"""
    
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        # Tip
        pass

    def forward(self, x, adj):
        # Tip
        pass

        
class GCN(nn.Module):
    '''GCN model with two GCN layers'''
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        # Tip
        pass

    def forward(self, x, adj):
        # Tip
        pass


def train(epoch):
    '''Train the model on the training data'''
    
    t = time.time()
    model.train()
    optimizer.zero_grad()
    
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # Evaluate the model on the val data
    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    print('Epoch: {:04d}'.format(epoch + 1),
        'loss_train: {:.4f}'.format(loss_train.item()),
        'acc_train: {:.4f}'.format(acc_train.item()),
        'loss_val: {:.4f}'.format(loss_val.item()),
        'acc_val: {:.4f}'.format(acc_val.item()),
        'time: {:.4f}s'.format(time.time() - t))

    
def test():
    '''Evaluate the model on the testing data'''
    
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:", "loss= {:.4f}".format(loss_test.item()), "accuracy= {:.4f}".format(acc_test.item()))

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.05,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=32,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args = parser.parse_args(args=[])
args.cuda = torch.cuda.is_available()
if args.cuda:
    torch.cuda.manual_seed(42)

# Model and optimizer
model = GCN(nfeat=features.shape[1], nhid=args.hidden, nclass=labels.max().item() + 1, dropout=args.dropout)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

# Load data into CUDA
if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")

# Testing
test()